In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import cftime
import calendar
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import matplotlib.ticker as mticker
from matplotlib.colors import TwoSlopeNorm
import imageio
import os
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from datetime import datetime

In [2]:
window_before = 5
window_after = 10

In [3]:
t2m_all_forcing_tes1 = xr.open_dataset('/nird/datalake/NS1004K/elihho/tes0004_echam6_BOT_mm_0_1850_var167.nc')
t2m_all_forcing_slo1 = xr.open_dataset('/nird/datalake/NS1004K/elihho/slo0059_echam6_BOT_mm_0_1850_var167.nc')

tas_orb_GHG = xr.open_dataset('slo0043_echam6_BOT_mm_1001-8850_167_NH.nc')
tas_all_forcing = xr.open_dataset('slo0042+slo0046+slo0050_echam6_BOT_mm_1001_8850_167_NH.nc')
tas_orb_GHG = tas_orb_GHG.sel(time=slice(70010131.994444, 88501231.994444))
tas_all_forcing = tas_all_forcing.sel(time=slice(70010131.994444, 88501231.994444))

In [4]:
#Function to convert time to date time object

#Split up in yyyy, mm and dd lists
def convert_time(time_array):
    #Converting time to pandas dataframe
    time_df = pd.DataFrame(time_array)#-0.99444445

    #Create empty lists
    time = []
    year = []
    month = []
    date = []
    
    #Split up date format in year, month and day
    for i in range(0,len(time_df[0])):
        time.append(str(time_df[0][i]))
        year.append(int(time[i][0:4])-7001)
        month.append(time[i][4:6])
    
    #Add lists to dataframe
    time_df['year']=year
    time_df['month']=month
    #Get the right format (str, yyyy, mm)
    yyyy=time_df['year'].astype("string").str.zfill(4)
    mm=time_df['month'].astype("string")

    #Combine to datestring in format 'yyyy-mm'
    for i in range(0,len(time_df[0])):
        date_stamp = f'{yyyy[i]}-{mm[i]}'
        date.append(date_stamp)

    #Add date string to dataframe
    time_df['date'] = date
    
    #Convert to datetime object. Works from year 0001!
    def convert_to_datetime(date_str):
        try:
            return datetime.strptime(date_str, '%Y-%m')
        except ValueError as e:
            print(f"Error converting date {date_str}: {e}")
            return pd.NaT
    
    # Apply the conversion function to the 'date' column
    time_df['date'] = time_df['date'].apply(convert_to_datetime)
    return time_df['date']

In [5]:
time_t2m_tes = convert_time(t2m_all_forcing_tes1['time'])
time_t2m_slo = convert_time(t2m_all_forcing_slo1['time'])
time_tas_orb_GHG = convert_time(tas_orb_GHG['time'])
time_tas_all_forcing = convert_time(tas_all_forcing['time'])

Error converting date 0000-01: year 0 is out of range
Error converting date 0000-02: year 0 is out of range
Error converting date 0000-03: year 0 is out of range
Error converting date 0000-04: year 0 is out of range
Error converting date 0000-05: year 0 is out of range
Error converting date 0000-06: year 0 is out of range
Error converting date 0000-07: year 0 is out of range
Error converting date 0000-08: year 0 is out of range
Error converting date 0000-09: year 0 is out of range
Error converting date 0000-10: year 0 is out of range
Error converting date 0000-11: year 0 is out of range
Error converting date 0000-12: year 0 is out of range
Error converting date 0000-01: year 0 is out of range
Error converting date 0000-02: year 0 is out of range
Error converting date 0000-03: year 0 is out of range
Error converting date 0000-04: year 0 is out of range
Error converting date 0000-05: year 0 is out of range
Error converting date 0000-06: year 0 is out of range
Error converting date 0000-0

In [6]:
t2m_tes = t2m_all_forcing_tes1.assign_coords(time = ('time', list(time_t2m_tes)))
t2m_slo = t2m_all_forcing_slo1.assign_coords(time = ('time', list(time_t2m_slo)))
tas_orb_GHG = tas_orb_GHG.assign_coords(time = ('time', list(time_tas_orb_GHG)))
tas_all_forcing = tas_all_forcing.assign_coords(time = ('time', list(time_tas_all_forcing)))

In [7]:
t2m_tes_lia = t2m_tes.sel(time=slice(datetime(1250, 1, 1, 0, 0), datetime(1849, 11, 1, 0, 0)))
t2m_slo_lia = t2m_slo.sel(time=slice(datetime(1250, 1, 1, 0, 0), datetime(1849, 11, 1, 0, 0)))

tas_orb_GHG_lia = tas_orb_GHG.sel(time=slice(datetime(1250, 1, 1, 0, 0), datetime(1849, 11, 1, 0, 0)))
tas_all_forcing_lia = tas_all_forcing.sel(time=slice(datetime(1250, 1, 1, 0, 0), datetime(1849, 11, 1, 0, 0)))

In [8]:
# Extract latitude
lat = t2m_tes_lia['lat']

# Calculate weights: cos(lat) in radians
weights = np.cos(np.deg2rad(lat))

scand_lat = slice(89, 50)
scand_lon = slice(-10, 50)

scand_lia_tes = t2m_tes_lia['var167'].sel(lat=scand_lat, lon = scand_lon).weighted(weights).mean(dim=['lat', 'lon'])
scand_lia_slo = t2m_slo_lia['var167'].sel(lat=scand_lat, lon = scand_lon).weighted(weights).mean(dim=['lat', 'lon'])
scand_orb_GHG_lia = tas_orb_GHG_lia['var167'].sel(lat=scand_lat, lon = scand_lon).weighted(weights).mean(dim=['lat', 'lon'])
scand_all_forcing_lia = tas_all_forcing_lia['var167'].sel(lat=scand_lat, lon = scand_lon).weighted(weights).mean(dim=['lat', 'lon'])

rolling_year=10
rolling_months = rolling_year*12
# Smooth with rolling mean
scand_smoothed_tes = scand_lia_tes.rolling(time=rolling_months, center=True).mean()
scand_smoothed_slo = scand_lia_slo.rolling(time=rolling_months, center=True).mean()
scand_orb_GHG_smooth = scand_orb_GHG_lia.rolling(time=rolling_months, center=True).mean()
scand_all_forcing_smooth = scand_all_forcing_lia.rolling(time=rolling_months, center=True).mean()

scand_lia_tes

<xarray.DataArray 'var167' (time: 6959)> Size: 56kB
array([261.76245641, 263.65661208, 265.92674976, ..., 279.05237829,
       275.66681056, 270.11839351])
Coordinates:
  * time     (time) object 56kB 1250-01-01 1250-02-01 ... 1849-10-01 1849-11-01

In [9]:
def calculate_yearly_mean(array):
    # Assuming 'time' contains datetime.datetime objects
    years = [t.year for t in array['time'].values]
    months = [t.month for t in array['time'].values]
    # Add these as coordinates to your DataArray
    array['year'] = ('time', years)
    array['month'] = ('time', months)

    yearly_mean_data = array.groupby('year').mean(dim='time')
    return yearly_mean_data

In [12]:
eruptions = [1257, 1477, 1510, 1585, 1595, 1640, 1667, 1673, 1707, 1721, 1739, 1755, 1766, 1783, 1815, 1822, 1831]

In [10]:
def superposed_epoch_analysis(series, eruption_years):
    window = np.arange(-5, 10 + 1)
    all_events = []

    for year in eruption_years:
        try:
            event_window = series.loc[year - window_before : year + window_after]
            if len(event_window) == len(window):
                all_events.append(event_window.values)
        except KeyError:
            continue  # Skip events that fall outside the time range

    if not all_events:
        raise ValueError("No valid eruption years within range.")
    
    stacked = np.vstack(all_events)
    mean_response = stacked.mean(axis=0)
    std_response = stacked.std(axis=0)

    return window, mean_response, std_response


In [11]:
window, mean_response, std_response = superposed_epoch_analysis(
    temps_by_year,
    eruption_years,
    window_before=5,
    window_after=10
)

NameError: name 'temps_by_year' is not defined

In [ ]:
rel_years, mean_anomaly, std_anomaly, all_epochs = superposed_epoch_analysis_simple(
    years_clean, wf_values_clean, eruption_years, window_before=5, window_after=10
)
